<a href="https://colab.research.google.com/github/goelnikhils-lgtm/languagemodels/blob/main/Fine_Tuning_Code_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#fine tuning - Instruct Fine Tuning Code LLM  - SFT
#CREDIT - >https://milvus.io/ai-quick-reference/what-is-the-learning-rate-schedule-used-during-finetuning - learning rate for larger models can be range between 1e-5 to 5e-5..
#.. this will help the model to not distrub too much of pre-trained weights
#warm up steps are typically 10% of overall training steps however warm up steps can be more for bigger models
#idea is that model should not get stuck in local minima
#optimizer like AdamW can be used and annealing rate can be cosine etc .... and move to 0
#FINE TUNING CODE CREDIT - https://www.youtube.com/watch?v=CUJexhbvBqM and BIGCODER https://github.com/bigcode-project/starcoder/blob/main/finetune/finetune.py
#CREDIT - >https://github.com/pacman100/LLM-Workshop/blob/main/chat_assistant/dpo/utils.py
#CREDIT - >https://huggingface.co/blog/personal-copilot


In [ ]:
!pip install torch torchvision torchaudio
!pip install transformers peft accelerate bitsandbytes datasets wandb

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, GenerationConfig
import torch
import wandb

In [ ]:
"""checkpoint = "bigcode/starcoder"
device = "cuda"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint, trust_remote_code=True).to(device)

inputs = tokenizer("def print_hello_world():", return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
"""

In [ ]:
#!git clone https://github.com/bigcode-project/starcoder.git

In [ ]:
#!python /content/starcoder/finetune/finetune.py

In [ ]:
#fine tuning code from starcoder
import argparse
import os
import torch
from accelerate import Accelerator
from datasets import load_dataset
from peft import LoraConfig, get_peft_model,  prepare_model_for_kbit_training , set_peft_model_state_dict
from torch.utils.data import IterableDataset
from tqdm import tqdm

from transformers import AutoConfig , AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments , Trainer , logging , set_seed
from transformers import TrainerCallback , TrainingArguments, TrainerState , TrainerControl

from transformers.trainer_utils import PREFIX_CHECKPOINT_DIR

"""
Fine tune Starcoder on Alpaca Dataset
"""
class SavePeftModelCallback(TrainerCallback):
    def on_save(self, args: TrainingArguments, state: TrainerState, control: TrainerControl, **kwargs):
        checkpoint_folder = os.path.join(args.output_dir, f"{PREFIX_CHECKPOINT_DIR}-{state.global_step}")
        kwargs["model"].save_pretrained(checkpoint_folder)
        pytorch_model_path = os.path.join(checkpoint_folder, 'pytorch_model.bin')
        torch.save({}, pytorch_model_path)
        return control

class LoadBestPeftModelCallback(TrainerCallback):
    def on_train_end(self, args: TrainingArguments, state: TrainerState, control: TrainerControl, **kwargs):
      print(f"Loading best model from {state.best_model_checkpoint} (score:{state.best_metric})")
      best_model_path = os.path.join(state.best_model_checkpoint, 'adapter_model.bin')
      adapter_weights = torch.load(best_model_path)
      model = kwargs['model']
      set_peft_model_state_dict(model, adapter_weights)
      return control

def get_args():
  parser = argparse.ArgumentParser()
  parser.add_argument("--model_name", type=str, default="bigcode/starcoder")
  parser.add_argument("--dataset_name", type=str, default="HuggingFaceH4/CodeAlpaca-20k")
  parser.add_argument("--subset", type=str)
  parser.add_argument("--split",type = str)
  parser.add_argument("--size_valid_set" , type = int,default = 10000)
  parser.add_argument("--size_train_set" , type = int, default = 10000)
  parser.add_argument("--streaming", action="store_true")
  parser.add_argument("--shuffle_buffer",type = int , default = 5000)
  parser.add_argument("--input_column", type = str, default = "prompt")
  parser.add_argument("--output_column", type = str, default = "completion")

  parser.add_argument("--seq_length",type = int , default = 2048)
  parser.add_argument("--max_steps",type=int,default = 10000)
  parser.add_argument("--batch_size",type = int , default = 4)
  parser.add_argument("--gradient_accumulation_steps",type = int , default = 16)
  parser.add_argument("--eos_token_id",type = int , default = 41952)

  parser.add_argument("--lora_r",type = int , default = 16)
  parser.add_argument("--lora_alpha",type = int , default = 32)
  parser.add_argument("--lora_dropout",type = float , default = 0.05)
  parser.add_argument("--lora_target_modules",type = str , default = "query_key_value") #this is key


  parser.add_argument("--learning_rate",type = float , default = 5e-6) # this is key start with lower
  parser.add_argument("--lr_scheduler_type",type = str , default = "cosine") # this is key cosine
  parser.add_argument("--num_warmup_steps",type = int , default = 100)
  parser.add_argument("--weight_decay",type = float , default = 0.05)

  parser.add_argument("--local_rank",type = int , default = 0)
  parser.add_argument("--no_fp16",action = "store_false")
  parser.add_argument("--bf16",action = "store_true", default = True)
  parser.add_argument("--no_gradient_checkpointing",action = "store_false",default = False)
  parser.add_argument("--seed",type = int , default = 0)
  parser.add_argument("--num_workers",type = int , default = None)
  parser.add_argument("--output_dir", type=str, default="./checkpoints")
  parser.add_argument("--log_freq",type = int , default = 100)
  parser.add_argument("--eval_freq",type = int , default = 100)
  parser.add_argument("--save_freq",type = int , default = 1000)
  args, unknown = parser.parse_known_args()
  return args

def chars_token_ratio(dataset,tokenizer,input_column_name = "prompt",output_column_name = "completion",nb_examples = 400):
  """
  Estimate the average number of characters per token in the dataset.
  """
  total_characters, total_tokens = 0,0
  for _ , example in tqdm(zip(range(nb_examples),iter(dataset)),total = nb_examples):
    text = prepare_sample_text(example,input_column_name, output_column_name)
    total_characters += len(text)
    if tokenizer.is_fast:
      total_tokens += len(tokenizer(text).tokens())
    else:
      total_tokens += len(tokenizer.tokenize(text))
  return total_characters / total_tokens

def print_trainable_parameters(model):
  """
  Prints the number of trainable parameters in the model.
  """
  trainable_params = 0
  all_param = 0
  for _, param in model.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
      trainable_params += param.numel()
  print(
      f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
  )
def prepare_sample_text(example ,input_column_name = "prompt",output_column_name = "completion"):
  """
  Prepare the text from a sample of the dataset.
  """
  text = f"{example[input_column_name]}\n\nAnswer {example[output_column_name]}"
  return text

class ConstantLengthDataset(IterableDataset):
  """
  Iterable dataset that returns constant length chunks of tokens from stream of text files
  Args:
  tokenizer
  dataset
  infinite
  seq_length - length  of token sequences to return
  num_of_sequences - number of sequences to keep in buffer
  chars_per_token  - no of characters per token used to estimate number of token in text buffer
  """
  def __init__(self,tokenizer,dataset,infinite = False ,seq_length = 1024,num_of_sequences = 1024,chars_per_token = 3.6 ,
               input_column_name = "prompt",output_column_name = "completion"):
    self.tokenizer = tokenizer
    self.concat_token_id = tokenizer.eos_token_id if tokenizer.eos_token_id is not None else args.eos_token_id
    self.dataset = dataset
    self.infinite = infinite
    self.seq_length = seq_length
    self.current_size = 0
    self.max_buffer_size = seq_length * num_of_sequences * chars_per_token
    self.input_column_name = input_column_name
    self.output_column_name = output_column_name

  def __iter__(self):
    iterator = iter(self.dataset)
    more_examples = True
    while more_examples:
      buffer,buffer_len = [],0
      while True:
        if buffer_len >= self.max_buffer_size:
          break
        try:
          buffer.append(prepare_sample_text(next(iterator),self.input_column_name,self.output_column_name))
          buffer_len += len(buffer[-1])
        except StopIteration:
          if self.infinite:
            iterator = iter(self.dataset)
          else:
            more_examples = False
            break
      tokenized_inputs = self.tokenizer(buffer,truncation = False)["input_ids"]
      all_token_ids =[]
      for tokenized_input in tokenized_inputs:
        all_token_ids.extend(tokenized_input + [self.concat_token_id]) #checking for eos token and appending that
      for i in range(0,len(all_token_ids),self.seq_length): # do till sequence length
        input_ids = all_token_ids[i:i+self.seq_length]
        if len(input_ids) == self.seq_length:
          self.current_size += 1
          yield {
            "input_ids":torch.LongTensor(input_ids),
            "labels":torch.LongTensor(input_ids)
            }
def create_dataset(tokenizer,args):
  dataset = load_dataset(args.dataset_name,
                         data_dir = args.subset,
                         split = args.split,
                         use_auth_token = True,
                         num_proc = args.num_workers if not args.streaming else None,
                         streaming = args.streaming
                         )
  if args.streaming:
    valid_data = dataset.take(args.size_valid_set)
    train_data = dataset.skip(args.size_valid_set)
    train_data = train_data.shuffle(buffer_size = args.shuffle_buffer , seed = args.seed)
  else:
    train_data = dataset["train"]
    valid_data = dataset["valid"]
  chars_per_token = chars_token_ratio(train_data,tokenizer,args.input_column,args.output_column)
  train_dataset = ConstantLengthDataset(tokenizer,
                                     train_data,
                                     infinite = True,
                                     seq_length = args.seq_length,
                                     chars_per_token = chars_per_token,
                                     input_column_name = args.input_column,
                                     output_column_name = args.output_column)

  valid_dataset = ConstantLengthDataset(tokenizer,
                                     valid_data,
                                     infinite = True,
                                     seq_length = args.seq_length,
                                     chars_per_token = chars_per_token,
                                     input_column_name = args.input_column,
                                     output_column_name = args.output_column)
  return train_dataset,valid_dataset
def run_training(args,train_dataset,valid_dataset):
  model = AutoModelForCausalLM.from_pretrained(args.model_name,use_auth_token = True,
                                               use_cache = not args.no_gradient_checkpointing,load_in_8bit = True,device_map = {"": Accelerator().process_index})

  model = prepare_model_for_kbit_training(model)
  lora_config = LoraConfig(
      r = args.lora_r,
      lora_alpha = args.lora_alpha,
      lora_dropout = args.lora_dropout,
      bias = "none",
      task_type = "CAUSAL_LM",
      target_modules = ["c_proj","c_attn","q_attn"])
  model = get_peft_model(model,lora_config)
  train_dataset.start_iteration = 0
  training_args = TrainingArguments(
          output_dir=args.output_dir,
          dataloader_drop_last=True,
          evaluation_strategy="steps",
          save_strategy="steps",
          load_best_model_at_end=True,
          max_steps=args.max_steps,
          eval_steps=args.eval_freq,
          save_steps=args.save_freq,
          logging_steps=args.log_freq,
          per_device_train_batch_size=args.batch_size,
          per_device_eval_batch_size=args.batch_size,
          learning_rate=args.learning_rate,
          lr_scheduler_type=args.lr_scheduler_type,
          warmup_steps=args.num_warmup_steps,
          gradient_accumulation_steps=args.gradient_accumulation_steps,
          gradient_checkpointing=not args.no_gradient_checkpointing,
          fp16=not args.no_fp16,
          bf16=args.bf16,
          weight_decay=args.weight_decay,
          run_name="StarCoder-finetuned",
          report_to="wandb",
          ddp_find_unused_parameters=False,
      )
  trainer = Trainer(model = model , args = training_args,
    train_dataset = train_dataset,
      eval_dataset = valid_dataset,
      callbacks = [SavePeftModelCallback,LoadBestPeftModelCallback])
  trainer.train()
  model.save_pretrained(args.output_dir)
  tokenizer.save_pretrained(args.output_dir)
def main(args):
  tokenizer = AutoTokenizer.from_pretrained(args.model_name,use_auth_token = True)
  train_dataset,eval_dataset = create_dataset(tokenizer,args)
  run_training(args,train_dataset,eval_dataset)
if __name__ == "__main__":
  args = get_args()
  set_seed(args.seed)
  os.makedirs(args.output_dir,exist_ok = True)
  main(args)